# Dog Breed Identification — Transfer Learning (VGG19)

This notebook demonstrates end-to-end steps: dataset download (Oxford-IIIT Pet), preprocessing, transfer learning with VGG19, training, evaluation, Grad-CAM explainability, saving the model, and a Flask inference wrapper.

In [ ]:
# 1) Environment setup & imports
import os
import random
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tfimport os
import random
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers

print("TensorFlow version:", tf.__version__)
print("Keras version:", keras.__version__)

import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers
print('TF version:', tf.__version__)
print('Keras version:', keras.__version__)


ModuleNotFoundError: No module named 'numpy'

In [2]:
# 2) Reproducibility & GPU check
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
# tf.random.set_seed(SEED)  # uncomment if needed

print('GPUs:', tf.config.list_physical_devices('GPU'))


NameError: name 'np' is not defined

In [ ]:
# 3) Download & prepare dataset (Oxford-IIIT Pet via tfds)
# We'll use tensorflow_datasets for a demo and prepare train/val/test splits as image datasets.
dataset_name = 'oxford_iiit_pet'
(ds_all, ds_info) = tfds.load(dataset_name, split='train+test', with_info=True, as_supervised=True)
print('Dataset info:', ds_info)

NUM_CLASSES = ds_info.features['label'].num_classes
print('Num classes (labels):', NUM_CLASSES)

# Convert to tf.data.Dataset with (image, label) pairs and simple preprocessing later in generators/cells


In [ ]:
# 4) Quick data inspection: show samples and class distribution
import math

sample_images = []
count = 0
for image, label in tfds.as_numpy(ds_all):
    sample_images.append((image, label))
    count += 1
    if count >= 12:
        break

plt.figure(figsize=(12,8))
for i, (img, lbl) in enumerate(sample_images):
    ax = plt.subplot(3,4,i+1)
    plt.imshow(img)
    plt.title(int(lbl))
    plt.axis('off')
plt.suptitle('Sample images (labels shown as integers)')
plt.show()


In [ ]:
# 5) Data preprocessing & augmentation (ImageDataGenerator compatible with VGG19)
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    brightness_range=[0.8,1.2],
    validation_split=0.2
)

# For demo we'll show how to use flow_from_directory later when you have folders organized.
print('Prepared ImageDataGenerator for augmentation')


In [ ]:
# 6) Create train/validation generators (example assumes directory layout)
# If you prepared `data/train` and `data/validation` use the following pattern:
# train_generator = train_gen.flow_from_directory('data/train', target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='sparse')
# val_generator = train_gen.flow_from_directory('data/validation', target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='sparse')

print('When using local folders: use flow_from_directory to create generators with class indices mapping.')


In [ ]:
# 7) Build transfer-learning model with VGG19 base
from tensorflow.keras.applications import VGG19

base_model = VGG19(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE, 3))
base_model.trainable = False

inputs = layers.Input(shape=(*IMG_SIZE, 3))
x = tf.keras.applications.vgg19.preprocess_input(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.4)(x)
outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)
model = keras.Model(inputs, outputs)

model.summary()


In [ ]:
# 8) Add classification head & compile model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy', keras.metrics.TopKCategoricalAccuracy(k=3, name='top_3')]
)


In [ ]:
# 9) Training — top-layer training (fit with callbacks)
callbacks = [
    keras.callbacks.ModelCheckpoint('models/dogbreed_best.h5', save_best_only=True, monitor='val_accuracy'),
    keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(patience=3, factor=0.5)
]

# NOTE: Replace the following placeholders with your actual generators or tf.data datasets.
# history = model.fit(train_generator, validation_data=val_generator, epochs=10, callbacks=callbacks)

print('Run training using `model.fit(...)` with your prepared generators. Example in models/train.py.')


In [ ]:
# 10) Fine-tuning (unfreeze last VGG blocks + continue training)
# Example: unfreeze from block5
for layer in base_model.layers:
    if 'block5' in layer.name:
        layer.trainable = True

model.compile(optimizer=keras.optimizers.Adam(1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# model.fit(..., epochs=5, callbacks=callbacks)
print('Unfroze block5 for fine-tuning; recompile with lower LR and continue training.')


In [ ]:
# 11) Evaluation: evaluate on test dataset and compute confusion matrix
# Example placeholders (run after you have a trained model & test_generator)
# test_loss, test_acc = model.evaluate(test_generator)
# print('Test accuracy:', test_acc)

# Compute per-class metrics and confusion matrix using sklearn when you have y_true and y_pred arrays


In [ ]:
# 12) Prediction examples for scenarios
from tensorflow.keras.preprocessing import image

def predict_image_pill(img_path, model, class_names, top_k=3):
    img = image.load_img(img_path, target_size=IMG_SIZE)
    x = image.img_to_array(img)/255.0
    x = np.expand_dims(x, 0)
    preds = model.predict(x)[0]
    idx = np.argsort(preds)[::-1][:top_k]
    return [(class_names[i], float(preds[i])) for i in idx]

# Example usage (after training and saving labels):
# class_names = open('models/labels.txt').read().splitlines()
# print(predict_image_pill('some.jpg', model, class_names))


In [ ]:
# 13) Explainability: Grad-CAM visualization (simple implementation)
import cv2

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]
    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / (tf.math.reduce_max(heatmap) + 1e-8)
    return heatmap.numpy()

# Usage: resize heatmap and overlay on original image for visualization.


In [ ]:
# 14) Save & export model (.h5 and SavedModel)
# model.save('models/dogbreed.h5')
# model.save('models/saved_model')

# Optional: convert to TFLite
# converter = tf.lite.TFLiteConverter.from_keras_model(model)
# tflite_model = converter.convert()
# open('models/dogbreed.tflite','wb').write(tflite_model)
print('Use model.save(...) to write HDF5/SavedModel formats.')

In [ ]:
# 15) Flask inference wrapper: predict_image(file)
import numpy as np
from PIL import Image

def predict_image_file(img_path, model, labels, target_size=(224,224), top_k=3):
    img = Image.open(img_path).convert('RGB').resize(target_size)
    x = np.array(img)/255.0
    x = np.expand_dims(x, 0)
    preds = model.predict(x)[0]
    idx = np.argsort(preds)[::-1][:top_k]
    return [(labels[i], float(preds[i])) for i in idx]

print('Helper for Flask inference ready.')

In [ ]:
# 16) Local Flask app demo & request tests (example)
# Example curl / requests usage to test the running Flask server
# curl -F "file=@/path/to/dog.jpg" http://127.0.0.1:5000/predict

# Example python requests test
# import requests
# r = requests.post('http://127.0.0.1:5000/predict', files={'file': open('dog.jpg','rb')})
# print(r.status_code, r.text)
print('See app.py in the repository for the full Flask demo.')

In [ ]:
# 17) Unit tests for pipeline & prediction functions (pytest examples)
# Save this as tests/test_pipeline.py when ready
#
# def test_preprocess_shape():
#     img = Image.new('RGB', (300,300), color='white')
#     img.save('tmp.jpg')
#     out = predict_image_file('tmp.jpg', model, ['cls0'])
#     assert isinstance(out, list)
#
print('Add pytest tests under tests/ to validate preprocessing, generators and Flask endpoints.')

In [ ]:
# 18) Helper utilities & wrap-up
# Plot training history utility

def plot_history(history):
    if history is None:
        return
    plt.figure(figsize=(12,4))
    plt.subplot(1,2,1)
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history.get('val_loss', []), label='val_loss')
    plt.legend()
    plt.subplot(1,2,2)
    plt.plot(history.history.get('accuracy', []), label='acc')
    plt.plot(history.history.get('val_accuracy', []), label='val_acc')
    plt.legend()
    plt.show()

print('Notebook ready — follow cells to train and export your model.')

Notebook ready — follow cells to train and export your model.
